In [1]:
import warnings
warnings.filterwarnings('ignore')

# import preprocessing function
from helpers.preprocessing import *

# import dependencies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sqlalchemy import create_engine
import psycopg2
from config import db_password

# import machine learning dependencies
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# import neural network dependencies
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Read the Data from SQL

In [2]:
X, y = preprocessing('fall')
X.head()

,accommodates,longitude,latitude,beds,bedrooms,review_scores_rating,bathrooms,room_Entire home/apt,room_Hotel room,room_Private room,...,property_Entire condo,property_Entire cottage,property_Entire guest suite,property_Entire guesthouse,property_Entire home,property_Entire rental unit,property_Entire serviced apartment,property_Entire townhouse,property_Other,property_Private room in home
0,8,-117.25760,32.80751,3.0,2.0,4.76,2.0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,-117.21931,32.74217,1.0,1.0,4.33,1.0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,7,-117.25416,32.79783,5.0,3.0,4.75,2.5,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,8,-117.25728,32.80751,6.0,3.0,4.72,2.0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,3,-117.26856,32.81301,3.0,2.0,4.94,1.0,1,0,0,...,0,0,0,1,0,0,0,0,0,0


# Split the Data into Training and Testing

In [4]:
X.describe()

,accommodates,longitude,latitude,beds,bedrooms,review_scores_rating,bathrooms,room_Entire home/apt,room_Hotel room,room_Private room,...,property_Entire condo,property_Entire cottage,property_Entire guest suite,property_Entire guesthouse,property_Entire home,property_Entire rental unit,property_Entire serviced apartment,property_Entire townhouse,property_Other,property_Private room in home
count,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000,...,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000,14026.000000
mean,4.866391,-117.173776,32.763467,2.688222,1.943747,4.749504,1.603629,0.838372,0.001069,0.155497,...,0.163696,0.021959,0.038856,0.047555,0.269642,0.184372,0.021959,0.035577,0.138315,0.078069
std,3.133007,0.074927,0.096004,2.111636,1.204703,0.387233,0.984302,0.368123,0.032686,0.362391,...,0.370013,0.146556,0.193260,0.212829,0.443790,0.387801,0.146556,0.185239,0.345242,0.268290
min,1.000000,-117.282580,32.519860,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,-117.248773,32.719043,1.000000,1.000000,4.710000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,-117.166917,32.756855,2.000000,2.000000,4.820000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6.000000,-117.129102,32.798990,3.000000,2.750000,4.950000,2.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,16.000000,-116.896560,33.125690,50.000000,23.000000,5.000000,27.500000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
# split data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [6]:
# scale the X data
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Multiple Linear Regression Model

In [7]:
reg = LinearRegression()
reg.fit(X_train_scaled, y_train)
y_pred = reg.predict(X_test_scaled)

In [8]:
print(f'Training R^2: {reg.score(X_train_scaled, y_train)}')
print(f'Testing R^2: {reg.score(X_test_scaled, y_test)}')

Training R^2: 0.10776308248288724
Testing R^2: 0.016422702490497265


In [9]:
# Display the coefficients in the model
results_df = pd.DataFrame({'Values': X.columns, 'Coef':reg.coef_})
results_df

,Values,Coef
0,accommodates,9.793644
1,longitude,-46.086466
2,latitude,35.128502
3,beds,74.503464
4,bedrooms,37.324485
5,review_scores_rating,-11.184779
6,bathrooms,140.445089
7,room_Entire home/apt,3.033836
8,room_Hotel room,1.198153
9,room_Private room,-0.029727


## Neural Network

In [10]:
X_train_scaled.shape

(10519, 21)

In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = 21
hidden_nodes_layer1 = 63
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 63)                1386      
                                                                 
 dense_1 (Dense)             (None, 30)                1920      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 3,337
Trainable params: 3,337
Non-trainable params: 0
_________________________________________________________________


In [12]:
nn.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [13]:
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
329/329 [==============================] - 3s 3ms/step - loss: 162.0309
Epoch 2/100
329/329 [==============================] - 1s 3ms/step - loss: 155.9972
Epoch 3/100
329/329 [==============================] - 1s 3ms/step - loss: 151.9561
Epoch 4/100
329/329 [==============================] - 1s 3ms/step - loss: 151.3553
Epoch 5/100
329/329 [==============================] - 1s 3ms/step - loss: 150.1273
Epoch 6/100
329/329 [==============================] - 1s 3ms/step - loss: 150.3592
Epoch 7/100
329/329 [==============================] - 1s 3ms/step - loss: 150.7863
Epoch 8/100
329/329 [==============================] - 1s 3ms/step - loss: 150.4077
Epoch 9/100
329/329 [==============================] - 1s 3ms/step - loss: 148.9972
Epoch 10/100
329/329 [==============================] - 1s 3ms/step - loss: 149.1781
Epoch 11/100
329/329 [==============================] - 1s 3ms/step - loss: 148.9307
Epoch 12/100
329/329 [==============================] - 1s 3ms/step - loss

329/329 [==============================] - 1s 3ms/step - loss: 144.9619
Epoch 98/100
329/329 [==============================] - 1s 3ms/step - loss: 145.7186
Epoch 99/100
329/329 [==============================] - 1s 3ms/step - loss: 144.7855
Epoch 100/100
329/329 [==============================] - 1s 3ms/step - loss: 144.8620


In [14]:
# Evaluate the mean absolute error of the  model using the test data
mae = nn.evaluate(X_test_scaled,y_test,verbose=0)
print(f"Mean Absolute Error: {mae}")

Mean Absolute Error: 181.05654907226562
